# Aula 1

Definições básicas do agente

palavras chaves: Aula 1: "LANGCHAIN"

#### Definição do agente   

Este agente será um assistente para estudos, focado em leitura de papers.   
O propósito será explicar em linguagem simples o que está descrito no paper.   


Fluxo de atendimento do agente:



In [0]:
!pip install -q --upgrade -r requirements.txt

In [0]:
import json
import prompts
import os

# from langchain_google_genai import ChatGoogleGenerativeAI 
from databricks_langchain import ChatDatabricks
from pydantic import BaseModel, Field
from typing import Literal, List, Dict
from langchain_core.messages import SystemMessage, HumanMessage

# GEMINI_API_KEY = json.load(open("config.json"))["GEMINI_API_KEY"]

In [0]:
# llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature = 0, api_key = GEMINI_API_KEY)
llm = ChatDatabricks(endpoint = "databricks-llama-4-maverick", temperature = 0,  max_tokens = 8192)

In [0]:
class TriagemOut(BaseModel):
    decisao: Literal["PEDIDO_AJUDA", "TUDO_CERTO", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory = list)

In [0]:
# llm_triagem = ChatGoogleGenerativeAI(model = "databricks-llama-4-maverick", temperature = 0, api_key = GEMINI_API_KEY)
llm_triagem = ChatDatabricks(endpoint = "databricks-llama-4-maverick", temperature = 0, max_tokens = 8192)

In [0]:
triagem_prompt = """Você é o recepcionista virtual de uma sistema de estudo online chamado ReadMyPaper. Você deve fazer o primeiro atendimento dos usuários identificando se os requisitos para iniciar a leitura do paper foram cumpridos.
Dada a mensagem do usuário, retorne SOMENTE um JSON com:
'''{
"decisao": "PEDIDO_AJUDA" | "TUDO_CERTO" | "PEDIR_INFO" | "ABRIR_CHAMADO",
"urgencia": "BAIXA" | "BAIXA" | "MEDIA" | "ALTA",
"campos_faltantes": ["..."]
}'''
Regras:
- **PEDIDO_AJUDA**: Pedido de ajuda para enviar o paper para análise (Ex: "Como faço para enviar o paper?", "Como faço o upload do paper?).
- **TUDO_CERTO**: Perguntas claras sobre o conteúdo do paper (Ex: "Qual o principal assunto do paper?", "Qual a metodologia utilizada?").
- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o paper (Ex: "Preciso de ajuda", "Tenho uma dúvida geral", "Qualquer pergunta fora do contexto do ReadMyPaper").
- **ABRIR_CHAMADO**: Situações em que não é possível continuar com a atendimento com o usuário. (Ex: "O usuáiro afirma que já fez o envio do paper mas ainda não há conteúdo para ser avaliado.", "O usuário está com problemas ao tentar enviar o paper", "O usuário pede para abrir chamado").
Analise a mensagem e decida a ação mais apropriada.
"""

In [0]:
triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def get_papers_list():
    """
    
    """
    pass
          

def check_paper_exist(paper_name) -> list:
    """
    
    """
    if os.path.exists(f"papers_to_read"):
        return True
    return os.path.exists(f"papers/{paper_name}.pdf")
    


def triagem(message: str) -> dict:
    """
    Retorna um dicionário com as seguintes chaves:
    - decisao: "TUDO_CERTO", "PEDIR_INFO", "ABRIR_CHAMADO"
    - urgencia: "BAIXA", "MEDIA", "ALTA"
    - campos_faltantes: lista de campos faltantes
    """
           
    triagem_out: TriagemOut = triagem_chain.invoke([SystemMessage(content = triagem_prompt), HumanMessage(content = message)])

    return triagem_out.model_dump()  

In [0]:
testes = ["Qual o assunto principal do paper?", 
          "Preciso fazer o upload do paper",
          "O palmeiras tem mundial?",
          "Já enviei o paper, podemos começar?"]

for teste in testes:
  print(f"Pergunta do usuário: {teste}\nResposta do Agente: {triagem(teste)}")

# Aula 2

Construção de um RAG

palavras chaves: Aula 1: "LANGCHAIN"
palavras chaves: Aula 2: "CHUNKS"

In [0]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
from databricks_langchain import DatabricksEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [0]:
docs = []
for n in Path("papers").glob("*.pdf"):
    try:
        loader = PyMuPDFLoader(str(n))
        docs.extend(loader.load())
        print(f"Arquivo {n.name} carregado com sucesso!")

    except Exception as e:
        print(f"Error ao carregar o arquvio {n.name}: {e}")

In [0]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 25)
chuncks = splitter.split_documents(docs)

In [0]:
embeddings = DatabricksEmbeddings(endpoint = "databricks-bge-large-en")

In [0]:
vector_store = FAISS.from_documents(chuncks, embeddings)

In [0]:
retriever = vector_store.as_retriever(search_type = "similarity_score_threshold", search_kwargs = {"score_threshold": 0.5, "k": 5})

In [0]:
prompt_rag = ChatPromptTemplate.from_messages([ ("system", 
                                                """Você é um assistente de perguntas e respostas sobre o conteúdo do paper da empresa ReadMyPaper.
                                                    Responda SOMENTE com base no contexto fornecido.
                                                    Se você não souber a resposta, responda apenas; 'Não sei'."""),
                                                ("human", "Pergunta: {question}\n\nContexto:\n {context}")
                                                ])

In [0]:
documents_chain = create_stuff_documents_chain(llm = llm, prompt = prompt_rag)


In [0]:
def pergunta_rag(question: str) -> dict:
    """
    
    """
    docs = retriever.invoke(question)

    if not docs:
        return {"answer": "Não sei",
                "citacoes": [],
                "contexto_encontrado": False}
    
    answer = documents_chain.invoke({"question": question, "context": docs})
    
    txt = (answer or "").strip()
    
    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não",
                "citacoes": [],
                "contexto_encontrado": False}
        

    return {"answer": txt,
            "citacoes": docs,
            "contexto_encontrado": True}
      

In [0]:
testes = ["Qual o assunto principal do paper?", 
          "Preciso fazer o upload do paper",
          "O palmeiras tem mundial?",
          "Já enviei o paper, podemos começar?"]

for teste in testes:
    resposta = pergunta_rag(teste)
    print(f"Pergunta: {teste}")
    print(f"Resposta: {resposta["answer"]}")

    if resposta["contexto_encontrado"]:
        print("Citacoes:")
        print(resposta["citacoes"])
        print("---------------------------------------------")
    